## **This code aims to generate PCA dataframes**

Remember that **PCA dataframes already contain the return column**

In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from factor_analyzer import Rotator

In [2]:
project_dir = "/home/jupyter-tfg2425paula/prediction_project_v3"
os.chdir(project_dir)

clean_data_dir = os.path.join(project_dir, "00_data/clean")
pca_data_dir = os.path.join(project_dir, "00_data/pca")

In [3]:
types_securities = ["options", "technical"]
stocks = ['AAPL', 'MSFT', 'AMZN', 'NVDA', 'SPX']
years = ["15y", "10y", "5y", "2y"]

In [4]:
def generate_rotated_pca_df(features_df, target_df):
    
    pca = PCA(n_components=0.90)
    pca_data = pca.fit_transform(features_df)
    num_components = pca.n_components_
    explained_variance = pca.explained_variance_ratio_

    print(f'Number of selected components: {num_components}')
    print(f'Explained variance by component: {explained_variance}')

    original_features = features_df.columns
    pca_components = pd.DataFrame(
        pca.components_, columns=original_features, index=[f'PC{i+1}' for i in range(num_components)])

    pca_loadings = pca_components.values

    # Rotation
    rotator = Rotator(method='oblimin')
    rotated_loadings = rotator.fit_transform(pca_loadings)
    rotated_pca_components = pd.DataFrame(
        rotated_loadings,
        columns=pca_components.columns,
        index=pca_components.index
    )

    rotated_pca_data = np.dot(features_df, rotated_loadings.T)

    rotated_pca_features_df = pd.DataFrame(
        rotated_pca_data,
        columns=[f'Rotated_PC{i+1}' for i in range(rotated_pca_data.shape[1])]
    )

    rotated_final_df = pd.concat([rotated_pca_features_df, target_df.reset_index(drop=True)], axis=1)
    
    return rotated_final_df

def scale_data(df):
    """
    Scales specified columns in a DataFrame using the specified scaling method.
    
    Args:
        df (pd.DataFrame): The input DataFrame.
        selected_cols (list): A list of column names to scale.
        scaling_method (str): The scaling method to use ("standard" or "minmax"). Default is "standard".
    
    Returns:
        pd.DataFrame: The DataFrame with specified columns scaled.
    """
    selected_scale_cols = [col for col in df.columns if col != "Target"]
    
    scaler = StandardScaler()
    df[selected_scale_cols] = df[selected_scale_cols].apply(pd.to_numeric, errors="coerce")
    df_scaled = df.copy()
    df[selected_scale_cols].replace([np.inf, -np.inf], np.nan, inplace=True)
    df[selected_scale_cols].fillna(df[selected_scale_cols].mean(), inplace=True)
    df_scaled[selected_scale_cols] = scaler.fit_transform(df[selected_scale_cols])
        
    return df_scaled


In [5]:
for security_type in types_securities:
    for stock in stocks:
        for period in years:
            clean_filepath = f"{security_type}/{stock}/{period}_data.csv"
            intial_df = pd.read_csv(os.path.join(clean_data_dir, clean_filepath))

            print(clean_filepath)
            scaled_df = scale_data(intial_df)
            target_df = scaled_df['Target']
            features_df = scaled_df.drop(columns = ["Date", "Target"])
            pca_df = generate_rotated_pca_df(features_df, target_df)
            
            output_folder = os.path.join(pca_data_dir, f"{security_type}/{stock}")
            os.makedirs(output_folder, exist_ok=True)
            
            pca_df.to_csv(os.path.join(pca_data_dir, clean_filepath), index=False)


options/AAPL/15y_data.csv
Number of selected components: 6
Explained variance by component: [0.31242667 0.232207   0.13005256 0.11050965 0.09119593 0.05821267]
options/AAPL/10y_data.csv
Number of selected components: 6
Explained variance by component: [0.31112435 0.25478101 0.11878127 0.0977161  0.0890532  0.0570205 ]


/tmp/ipykernel_13395/1377450272.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[selected_scale_cols].replace([np.inf, -np.inf], np.nan, inplace=True)
/tmp/ipykernel_13395/1377450272.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[selected_scale_cols].fillna(df[selected_scale_cols].mean(), inplace=True)
/home/jupyter-tfg2425paula/.local/lib/python3.9/site-packages/sklearn/utils/extmath.py:1137: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/home/jupyter-tfg2425paula/.local/lib/python3.9/site-packages/sklearn/utils/extmath.py:1142: Runtime

options/AAPL/5y_data.csv
Number of selected components: 6
Explained variance by component: [0.33518157 0.21447856 0.12933665 0.10219766 0.09113943 0.0563259 ]
options/AAPL/2y_data.csv
Number of selected components: 5
Explained variance by component: [0.38886097 0.18543289 0.13256468 0.12164312 0.08707735]
options/MSFT/15y_data.csv
Number of selected components: 5
Explained variance by component: [0.41526508 0.20370144 0.11840808 0.08886373 0.07949863]


/tmp/ipykernel_13395/1377450272.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[selected_scale_cols].replace([np.inf, -np.inf], np.nan, inplace=True)
/tmp/ipykernel_13395/1377450272.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[selected_scale_cols].fillna(df[selected_scale_cols].mean(), inplace=True)
/home/jupyter-tfg2425paula/.local/lib/python3.9/site-packages/sklearn/utils/extmath.py:1137: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/home/jupyter-tfg2425paula/.local/lib/python3.9/site-packages/sklearn/utils/extmath.py:1142: Runtime

options/MSFT/10y_data.csv
Number of selected components: 5
Explained variance by component: [0.50382051 0.1618703  0.10403398 0.08734723 0.07897683]
options/MSFT/5y_data.csv
Number of selected components: 5
Explained variance by component: [0.37680799 0.1919581  0.15024945 0.12340558 0.08547733]
options/MSFT/2y_data.csv


...

/tmp/ipykernel_13395/1377450272.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[selected_scale_cols].replace([np.inf, -np.inf], np.nan, inplace=True)
/tmp/ipykernel_13395/1377450272.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[selected_scale_cols].fillna(df[selected_scale_cols].mean(), inplace=True)
/home/jupyter-tfg2425paula/.local/lib/python3.9/site-packages/sklearn/utils/extmath.py:1137: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/home/jupyter-tfg2425paula/.local/lib/python3.9/site-packages/sklearn/utils/extmath.py:1142: Runtime

technical/SPX/5y_data.csv
Number of selected components: 22
Explained variance by component: [0.28119194 0.19030746 0.08275816 0.06277883 0.03675481 0.03196126
 0.03005609 0.02102131 0.01979548 0.01706286 0.01453111 0.01375637
 0.01245787 0.01176595 0.01130722 0.01079985 0.01000923 0.00983648
 0.00904339 0.00889447 0.0082289  0.00785315]


/tmp/ipykernel_13395/1377450272.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[selected_scale_cols].replace([np.inf, -np.inf], np.nan, inplace=True)
/tmp/ipykernel_13395/1377450272.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[selected_scale_cols].fillna(df[selected_scale_cols].mean(), inplace=True)
/home/jupyter-tfg2425paula/.local/lib/python3.9/site-packages/sklearn/utils/extmath.py:1137: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/home/jupyter-tfg2425paula/.local/lib/python3.9/site-packages/sklearn/utils/extmath.py:1142: Runtime

technical/SPX/2y_data.csv
Number of selected components: 21
Explained variance by component: [0.28603993 0.15856845 0.08960154 0.0800908  0.04868594 0.03080102
 0.02692267 0.02318216 0.01921273 0.01735148 0.01605376 0.01526851
 0.0135191  0.01296614 0.01121076 0.01073362 0.0103494  0.00931483
 0.00911052 0.0084809  0.00823079]


/tmp/ipykernel_13395/1377450272.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[selected_scale_cols].replace([np.inf, -np.inf], np.nan, inplace=True)
/tmp/ipykernel_13395/1377450272.py:55: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[selected_scale_cols].fillna(df[selected_scale_cols].mean(), inplace=True)
/home/jupyter-tfg2425paula/.local/lib/python3.9/site-packages/sklearn/utils/extmath.py:1137: RuntimeWarning: invalid value encountered in divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/home/jupyter-tfg2425paula/.local/lib/python3.9/site-packages/sklearn/utils/extmath.py:1142: Runtime

In [9]:
period

'2y'